Get the data into a data frame and check that it looks good. Note that our data has no headings row, so we furnish names for the columns


In [31]:
import pandas as pd
index=["Date","Time","Product","Quantity","OrderValue"]

df = pd.read_csv('SalesRecords.csv', names=index )
print(df.head())

         Date      Time        Product  Quantity OrderValue
0  20/08/2005  11:15 PM            Cod       1.0     £2.50 
1  31/12/2009   5:32 PM         Plaice       1.0     £2.90 
2  31/12/2009   5:34 PM        Sausage       2.0     £3.00 
3  31/12/2009   5:39 PM          Fanta       1.0     £0.65 
4  31/12/2009   5:41 PM  Chips (small)       4.0     £3.20 


Drop any rows with missing data. In a real case you would probably want to examine them, but for this example let's be rash. Notice use of `inplace` to mutate the existing data, default returns a copy. 

In [32]:
df.dropna(inplace=True)
df.tail()

,Date,Time,Product,Quantity,OrderValue
28497,31/12/2011,11:13 PM,Chips (large),2.0,£2.00
28498,31/12/2011,11:17 PM,Plaice,3.0,£8.70
28499,31/12/2011,11:24 PM,Haddock,1.0,£2.85
28500,1st May 2010,7:51 PM,Chips (small),2.0,£1.60
28501,22/08/0011,6:53 PM,Chips (small),2.0,£1.60


Let's add a categorical ProductType category to (eventually) replace the string Product. We'll print it to see the categories.

In [33]:
df["ProductType"]=df["Product"].astype("category")
print(df["ProductType"].cat.categories.tolist())

['COD', 'Chips', 'Chips (large)', 'Chips (small)', 'Cod', 'Coke', 'Fanta', 'Fish', 'Haddock', 'Place', 'Plaice', 'Pliace', 'Sausage', 'Soss']


Here we will use our knowledge to pick the 'right' categories, in a later example we will examine the numbers in each category.
We'll also drop the (string) Products category 

In [34]:
good_products = ['Chips (large)', 'Chips (small)', 'Cod', 'Coke', 'Fanta', 'Haddock', 'Place', 'Plaice', 'Sausage']
df = df[df["ProductType"].isin(good_products)]
df.drop(columns=['Product'], inplace=True)
print(df.tail())


               Date      Time  Quantity OrderValue    ProductType
28497    31/12/2011  11:13 PM       2.0     £2.00   Chips (large)
28498    31/12/2011  11:17 PM       3.0     £8.70          Plaice
28499    31/12/2011  11:24 PM       1.0     £2.85         Haddock
28500  1st May 2010   7:51 PM       2.0     £1.60   Chips (small)
28501    22/08/0011   6:53 PM       2.0     £1.60   Chips (small)


The OrderValue column has '£' signs in it and also variable spaces. We can use 'vector' string functions from pandas to operate on all cells without an explicit loop. Once we have done that we can attempt to convert from string to float. We'll drop any rows that fail to convert.

In [35]:
df["OrderValue"] = df["OrderValue"].str.strip(' £')
df["OrderValue"] = df["OrderValue"].astype(float, errors="ignore")
df.dropna(inplace=True)
print(df.tail())

               Date      Time  Quantity  OrderValue    ProductType
28497    31/12/2011  11:13 PM       2.0        2.00  Chips (large)
28498    31/12/2011  11:17 PM       3.0        8.70         Plaice
28499    31/12/2011  11:24 PM       1.0        2.85        Haddock
28500  1st May 2010   7:51 PM       2.0        1.60  Chips (small)
28501    22/08/0011   6:53 PM       2.0        1.60  Chips (small)


Similarly 'Quantity' should be an int

In [36]:
df["Quantity"]=df["Quantity"].astype(int)
print(df)

               Date      Time  Quantity  OrderValue    ProductType
0        20/08/2005  11:15 PM         1        2.50            Cod
1        31/12/2009   5:32 PM         1        2.90         Plaice
2        31/12/2009   5:34 PM         2        3.00        Sausage
3        31/12/2009   5:39 PM         1        0.65          Fanta
4        31/12/2009   5:41 PM         4        3.20  Chips (small)
...             ...       ...       ...         ...            ...
28497    31/12/2011  11:13 PM         2        2.00  Chips (large)
28498    31/12/2011  11:17 PM         3        8.70         Plaice
28499    31/12/2011  11:24 PM         1        2.85        Haddock
28500  1st May 2010   7:51 PM         2        1.60  Chips (small)
28501    22/08/0011   6:53 PM         2        1.60  Chips (small)

[28492 rows x 5 columns]


Now we have numbers, we can do arithmetic. Note use of 'groupby' and 'size' to find out how many are in each category

In [37]:
df["Check"]=(df["OrderValue"]/df["Quantity"]).astype("category")
print(df["Check"])
print(df.groupby("Check").size())

0        2.50
1        2.90
2        1.50
3        0.65
4        0.80
         ... 
28497    1.00
28498    2.90
28499    2.85
28500    0.80
28501    0.80
Name: Check, Length: 28492, dtype: category
Categories (17, float64): [0.100000, 0.228571, 0.600000, 0.650000, ..., 2.90, 3.75, 5.00, 285.00]
Check
0.1                      1
0.2285714285714286       1
0.6                   2154
0.65                  2000
0.7                      1
0.7999999999999999    1225
0.8                   3719
1.0                   5180
1.5                   1981
2.0                      1
2.5                   4038
2.6                      1
2.85                  4122
2.9                   4065
3.75                     1
5.0                      1
285.0                    1
dtype: int64
